In [1]:
## This script is developed for the purpose of visualization 
## of the FARGO output
## This script combines the FARGO recommended script and the multifluid script developed by me

# Author: sayantan
# Date : 9 September 2019
# modified : 



####### Importing the required modules

# from pylab import*
import matplotlib.pyplot as plt
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rc('text', usetex=True)
# from matplotlib import ticker, cm
import os
# import glob
# import time
import multiprocessing as mp
# import cv2

import numpy as np

In [2]:
## Making the Parameter class 
class Parameters(object):
    """
    Class for reading the simulation parameters.
    input: string -> name of the parfile, normally variables.par
    """
    def __init__(self, paramfile):
        try:
            params = open(paramfile,'r') #Opening the parfile
        except IOError:                  # Error checker.
            print  (paramfile + " not found.")
            return
        lines = params.readlines()     # Reading the parfile
        params.close()                 # Closing the parfile
        par = {}                       # Allocating a dictionary
        for line in lines:             #Iterating over the parfile
            name, value = line.split() #Spliting the name and the value (first blank)
            try:
                float(value)           # First trying with float
            except ValueError:         # If it is not float
                try:
                    int(value)         #                   we try with integer
                except ValueError:     # If it is not integer, we know it is string
                    value = '"' + value + '"'
            par[name] = value          # Filling the dictory
        self._params = par             # A control atribute, actually not used, good for debbuging
        for name in par:               # Iterating over the dictionary
            exec("self."+name.lower()+"="+par[name]) #Making the atributes at runtime




In [3]:
## Making a Visualization Class 

class fargo_visualization(object):
    
    """
    Class for reading the simulation parameters.
    input: string -> name of the output folder, normally fargo_multifluyid.par
          : string -> path the output folder
    """
    
    def __init__(self,output_folder):
#         super().__init__(paramfile)
                
        self.output_folder = output_folder  
        paramfile = output_folder +"variables.par"
                
#         print(paramfile)
        print(self.output_folder)
        self.P = Parameters(paramfile)
        print (self.P.nx, self.P.ny, self.P.omegaframe, self.P.outputdir, self.P.sigmaslope, self.P.sigma0)
        
        self.number_of_orbit = int(self.P.ntot/self.P.ninterm)
        
        ## Setting up the coordinates 
        
        self.xmin = np.loadtxt(output_folder+"domain_x.dat")       #face-centered coodinates
        self.ymin = np.loadtxt(output_folder+"domain_y.dat")[3:-3] #face-centered coordinates. Ghosts are included, so we remove them from the array
        self.xc   = 0.5*(self.xmin[1:]+self.xmin[:-1]) #cell-centered
        self.yc   = 0.5*(self.ymin[1:]+self.ymin[:-1]) #cell-centered


        
    def _density_plot(self,fluid,fluid_property,orbit= None,plot_type=None):
        
        ############## Creating output folders to save the output images##########
        
        current_directory = os.getcwd() 
        # print("the current directory in", current_directory)
        path= current_directory + '/movie_outputs_'+ fluid +'_'+ fluid_property
        path2 = current_directory + '/movie_folder'
        
        
        try:      
            os.makedirs(path)
        except OSError:  
            pass
#              print ("Creation of the directory %s failed/ not needed as it already exit" % path)
        else:  
            print ("Successfully created the directory %s" % path)

        #######################################################
        
#         print(plot_type)
        intial_file = self.output_folder +'/' + fluid + fluid_property +str(0)+".dat"
        
        initial_density = np.fromfile(intial_file).reshape(self.P.ny,self.P.nx) ## initial gas density P.ny * P.nx
        mean_intial_density = initial_density.mean(axis=1)
        
        if orbit == None:
            orbit = self.number_of_orbit ## this automatically picks the last orbit
        else: 
            orbit = orbit
        
        filename = self.output_folder +'/' + fluid + fluid_property + str(orbit)+".dat"
            
        density = np.fromfile(filename).reshape(self.P.ny,self.P.nx) ## evolving density 
        mean_density=density.mean(axis=1)
        
        fig = plt.figure()
        outputfilename_figfile = os.path.join(path, fluid +'_dens' +'%s'%orbit+ ".jpg")
        
        if plot_type == None or plot_type == "1D":
            plt.plot(self.yc,initial_density[:,0],label= "Initial %s"%fluid + " Density" + " res: %s"%self.P.ny +r"$\times$ %s"%self.P.nx )  
            plt.plot(self.yc,density[:,0],linestyle='--',label= " %s"%fluid + r" Density for a particular $\phi$" + " Orbit = %s"%orbit)  
    #             plt.plot(yc,mean_intial_density,linestyle='-',label= "Evolving %s"%fluid + " Density"+" Orbit = %s"%number) ## plotting the radial profile

            plt.axhline(y=1, color='r', linestyle='--')      
            plt.legend(loc=2)
            plt.yscale('log')

        
            plt.ylabel(r"Log $\Sigma$", fontsize=15)
        #         plt.ylabel(r"Log $\Delta \Sigma/\Sigma_{0}$", fontsize=15)
            plt.xlabel(r"Radius", fontsize=15)
            plt.tight_layout()
            plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
            plt.close()
            
        if plot_type == "2D":
            imshow(log10(density),origin='lower', aspect='auto',extent=[self.P.xmin,self.P.xmax,self.P.ymin,self.P.ymax])
#             plt.title("Fluid = %s"%fluid + "  Orbit = %s"%number+ " res: %s"%P.ny +r"$\times$ %s"%P.nx, fontsize=12,color='Black')
            plt.ylabel(r"Radius", fontsize=15)
            plt.xlabel(r"$\varphi$")
            plt.colorbar()
#             cbar.set_ylabel(r"Log $\Sigma$")
            plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
            plt.close()
            
            
        if plot_type == "2D_polar":
            r, theta = np.meshgrid(self.xc, self.yc)
        
            ax = plt.subplot(111, polar=True)
            ax.yaxis.grid(False)
            ax.xaxis.grid(False)
        #         cax = ax.contourf(r, theta, log10(param), 100,cmap='Blues')  
            cax = ax.contourf(r, theta, log10(density), 100 , cmap=plt.cm.Spectral)
            cbar = fig.colorbar(cax)
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            cbar.ax.set_ylabel(r"Log $\Sigma$")
            plt.savefig(outputfilename_figfile,format='jpg',dpi=300)
            plt.close()
#             plt.title("Fluid = %s"%fluid + "  Orbit = %s"%number+ " res: %s"%P.ny +r"$\times$ %s"%P.nx, fontsize=12,color='Black')
        
        
        return fig
    
#     def get_the_disk_gap(self,mean_param,yc,half_initial_gas_density):

#         radius_list = []
#         for index, (gas_dens, radius) in enumerate (zip(mean_param, yc)):        
#             if gas_dens <= half_initial_gas_density: ## for gas density less that that half the intial density
#                 radius_list.append(radius)


#         if len(radius_list) == 0: ## if there are no values below the half_initial_gas_density
#             radius_list= np.zeros(len(yc))

#         r_in  = min(radius_list)
#         r_out = max(radius_list)
#         gap = r_out - r_in      

#         return(gap)
    
    

    def _movie_density_plot(self,fluid,fluid_property,orbit= None,plot_type=None, frequency=None):
        
        if frequency == None:
            interval = 10 ## the default frequency
        else: 
            interval = frequency
        
        for number in range(0,self.number_of_orbit,interval):
            self._density_plot(fluid,fluid_property, orbit=number,plot_type=plot_type)
        
#         def do_plot(number):            
#             self._density_plot(fluid, orbit= number,plot_type=None)
    
#         try:      
#             os.makedirs(path2)
#         except OSError:  
#             print ("Creation of the directory %s failed/ not needed as it already exit" % path)
#         else:  
#             print ("Successfully created the directory %s" % path)


In [4]:
################################### User defined inputs ##################

## give to path to the directory of the output

folder ='fargo_multifluid_T0/'

fluid_list = ['gas','dust1']
fluid_property_list = ['dens','vy','vx']

#please select the particular fluid and the parameter from the list
f=1;p=0
fluid = fluid_list[f]
fluid_property= fluid_property_list[p]




V = fargo_visualization(output_folder=folder)



fargo_multifluid_T0/
384 128 0 ./outputs/fargo_multifluid_T0/ 0.5 0.0001


In [5]:
# V._density_plot(fluid,fluid_property,orbit=100)
# # V._density_plot(fluid,plot_type="2D")
# V._density_plot(fluid,fluid_property,plot_type="2D_polar",orbit=50)

        

In [6]:
V._movie_density_plot(fluid_list[0],fluid_property)
V._movie_density_plot(fluid_list[1],fluid_property)
